## Import Repository

In [1]:
!git clone https://github.com/federico2879/MLDL2024_semantic_segmentation.git

Cloning into 'MLDL2024_semantic_segmentation'...
remote: Enumerating objects: 454, done.
remote: Counting objects: 100% (105/105), done.
remote: Compressing objects: 100% (100/100), done.
remote: Total 454 (delta 55), reused 3 (delta 3), pack-reused 349
Receiving objects: 100% (454/454), 220.69 KiB | 3.06 MiB/s, done.
Resolving deltas: 100% (262/262), done.


## Import package

In [2]:
!pip install -U fvcore

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 705.4 kB/s eta 0:00:00 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 792.0 kB/s eta 0:00:00 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61400 sha256=3e3dbe467f2bd5f823f0722423e90518ec861a09015339c0388ccb33c7f0fa4e
  Stored in directory: /root/.cache/pip/wheels/01/c0/af/77c1cf53a1be9e42a52b48e5af2169d40ec2e89f7362489dd0
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31532 sha256=3f1571d79bb6a261077fec9d881110441baaf3e8f70d7c8f728a098b08691ef2
  Stored in directory: /root/.cache/pip/wheels/9a/a3/b6/ac0fcd1b4ed5cfeb3db92e6a0e476cfd48ed0df92b91080c1d
Successfully built fvcore iopath


In [3]:
import torch
from torch import nn
import torchvision
import torchvision.transforms as transforms
from torchvision.transforms.functional import InterpolationMode
from torch.utils.data import DataLoader
import numpy as np
import csv
from MLDL2024_semantic_segmentation.datasets.cityscapes import CityScapes
from MLDL2024_semantic_segmentation.models.deeplabv2.deeplabv2 import *
from MLDL2024_semantic_segmentation.train import * 
from MLDL2024_semantic_segmentation.utils import *
from MLDL2024_semantic_segmentation.models.metrics import * 
from MLDL2024_semantic_segmentation.models.IOU import * 

## Setup

In [4]:
# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

# Setup fixed parameters
num_classes = 19
num_epochs = 1

cuda


## Dataset

In [5]:
# Transformations
transform_image = transforms.Compose([
    transforms.Resize((1024, 512)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])
transform_target = transforms.Compose([
    transforms.Resize((1024, 512), interpolation=InterpolationMode.NEAREST)
])

In [6]:
# Create dataloader
train_dataset = CityScapes('/kaggle/input/cityscapes/Cityscapes/Cityspaces', split = 'train', transform = transform_image, label_transform = transform_target)
dataloader_train = DataLoader(train_dataset, batch_size=2, shuffle=True)

val_dataset = CityScapes('/kaggle/input/cityscapes/Cityscapes/Cityspaces', split = 'val', transform = transform_image, label_transform = transform_target)
dataloader_val = DataLoader(val_dataset, batch_size=2, shuffle=False)

## Network, Loss, Optimizer

In [7]:
# Pretrained
pretrained = torchvision.models.resnet101(pretrained=True).to(device)
torch.save(pretrained.state_dict(), "DeepLab_resnet_pretrained_imagenet.pth")

# Inizialization of the model
model = get_deeplab_v2().to(device)
diz_optim = model.optim_parameters(0.001) # parameters + learning rates

#Putting on the 2 gpus
model = torch.nn.DataParallel(model, device_ids = [0,1]).to(device)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth
100%|██████████| 171M/171M [00:01<00:00, 157MB/s]  


Deeplab pretraining loading...


In [8]:
# Define loss and optimizer
loss_fn = nn.CrossEntropyLoss(ignore_index=255)
optimizer = torch.optim.SGD(diz_optim, momentum=0.9, weight_decay=0.0005)

/opt/conda/lib/python3.10/site-packages/torch/_compile.py:24: UserWarning: optimizer contains a parameter group with duplicate parameters; in future, this will cause an error; see github.com/pytorch/pytorch/issues/40967 for more information
  return torch._dynamo.disable(fn, recursive)(*args, **kwargs)


## Training

In [ ]:
meanIOU_tr = np.zeros((num_epochs,1))
IOU_tr = np.zeros((num_epochs,num_classes))
loss_tr = np.zeros((num_epochs,1))


meanIOU_val = np.zeros((num_epochs,1))
IOU_val = np.zeros((num_epochs,num_classes))
loss_val = np.zeros((num_epochs,1))

In [9]:
# Set the random seeds
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [ ]:
for epoch in range(num_epochs):
    lr_1x = poly_lr_scheduler(optimizer, 0.001, epoch, lr_decay_iter=1,
                      max_iter=num_epochs, power=0.9)
    optimizer.param_groups[1]['lr'] = 10*lr_1x
    meanIOU_tr[epoch], IOU_tr[epoch,:], loss_tr[epoch] = train(model, optimizer, dataloader_train, loss_fn, num_classes, 0)
    meanIOU_val[epoch], IOU_val[epoch,:], loss_val[epoch] = test(model, dataloader_val, loss_fn, num_classes, 0)
    print(f"epoch: {epoch + 1}, Validation IOU: {meanIOU_val[epoch]:.2f}")

    torch.save({
        'epoch': epoch,
        'state_dict': model.state_dict(),
        'optimizer': optimizer.state_dict(),
        'val_IOU': meanIOU_val[epoch]
    },"checkpoint.pth.tar")

print(f"Final mIOU: {meanIOU_val[epoch]:.2f}")
print("Final IOU classes")
print(IOU_val[epoch,:])

flops = Flops(model, 1024, 512)

#print(f"Number of flops?: {flops}")

latency = Latency_FPS(model, 1024, 512)

print(f"Latency: {latency}")

#print(f"number of parameters: {model.count_params()}")

# Access the actual model being parallelized
actual_model = model.module
# Count the parameters of the actual model
num_params = count_params(actual_model)
print(f"number of parameters: {num_params}")

In [ ]:
with open('meanIOU_tr.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(meanIOU_tr)

with open('IOU_tr.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(IOU_tr)

with open('loss_tr.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(loss_tr)

with open('meanIOU_val.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(meanIOU_val)

with open('IOU_val.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(IOU_val)

with open('loss_val.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(loss_val)